# Capstone Phase 1: AI-Powered Requirements & PRD Generation

**Project:** TripSplit - Group Travel Expense Manager

**Objective:** Use Large Language Models (LLMs) to transform our project idea into structured requirements, user stories, and a comprehensive Product Requirements Document (PRD).

**Estimated Time:** 90 minutes

---

## 📋 Before You Begin

**Prerequisites:**
1. ✅ Create a `.env` file in the project root with your `GOOGLE_API_KEY`
2. ✅ Get your API key from: https://aistudio.google.com/app/apikey

**Format of `.env` file:**
```
GOOGLE_API_KEY=your_actual_api_key_here
```

---

## 📖 Introduction

This notebook combines the workflows from Day 1 Labs 1 and 2 to complete Phase 1 of our capstone project. We'll start with our TripSplit idea and use AI to generate:
1. Features and user personas
2. Structured user stories with acceptance criteria in JSON format
3. A formal Product Requirements Document (PRD)
4. A Pydantic validation model

All artifacts will be saved to the `Artifacts/Documentation/` folder.

**How to use this notebook:**
- Run **Step 0** first to install required packages
- Then run each cell in order from top to bottom


## Step 0: Install Required Packages

**Run this cell FIRST** to ensure all required packages are installed in the notebook's kernel environment.

This will install packages directly into the Python environment that Jupyter is using.


In [2]:
# Install required packages in the current kernel environment
import sys
import subprocess

print("Installing required packages in the current kernel environment...")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print("=" * 70)

# List of required packages for Phase 1
required_packages = [
    'python-dotenv',
    'google-genai',
    'pydantic',
    'ipython',
]

# Install each package
for package in required_packages:
    print(f"\n📦 Installing {package}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"   ✓ {package} installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Failed to install {package}: {e}")

print("\n" + "=" * 70)
print("✅ Package installation complete!")
print("\nYou can now proceed to Step 1.")
print("=" * 70)


Installing required packages in the current kernel environment...
Python executable: c:\Users\640109\OneDrive - BOOZ ALLEN HAMILTON\Documents\AISE_Capstone\220372-AG-AISOFTDEV-Team-1-AINavigators\.venv\Scripts\python.exe
Python version: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]

📦 Installing python-dotenv...
   ✓ python-dotenv installed successfully

📦 Installing google-genai...
   ✓ google-genai installed successfully

📦 Installing pydantic...
   ✓ pydantic installed successfully

📦 Installing ipython...
   ✓ ipython installed successfully

✅ Package installation complete!

You can now proceed to Step 1.


## Step 1: Setup

This cell performs the following setup tasks:

1. **Locate Project Root**: Find the project directory and add it to Python's path
2. **Load Environment Variables**: Load API keys from the `.env` file
3. **Verify Dependencies**: Check that required packages are installed
4. **Initialize LLM Client**: Set up the Google Gemini AI client

**Before running this cell, make sure:**
- ✅ You have installed dependencies: `pip install -r requirements.txt`
- ✅ You have created a `.env` file with your `GOOGLE_API_KEY`
- ✅ Your API key is from: https://aistudio.google.com/app/apikey

**Expected output:** You should see checkmarks (✓) for each step and a final success message.


In [3]:
import sys
import os
import json

# ============================================================================
# Step 1a: Find Project Root and Add to Path
# ============================================================================
# Get the project root directory (one level up from Python Notebooks)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"📁 Project root: {project_root}")

# Add project root to Python path so we can import utils
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# ============================================================================
# Step 1b: Load Environment Variables from .env
# ============================================================================
# Check if required packages are installed
try:
    from dotenv import load_dotenv
    print("✓ python-dotenv is installed")
except ImportError:
    print("❌ ERROR: python-dotenv is not installed!")
    print("   Run: pip install python-dotenv")
    raise

# Load the .env file from project root
env_path = os.path.join(project_root, '.env')
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"✓ Loaded .env file from: {env_path}")
else:
    print(f"⚠️  WARNING: .env file not found at: {env_path}")
    print("   Please create a .env file with your API keys.")
    print("   See SETUP.md for instructions.")

# Verify API key is loaded
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    # Only show first/last few characters for security
    masked_key = f"{google_api_key[:8]}...{google_api_key[-4:]}" if len(google_api_key) > 12 else "***"
    print(f"✓ GOOGLE_API_KEY found: {masked_key}")
else:
    print("❌ ERROR: GOOGLE_API_KEY not found in environment!")
    print("   Please add GOOGLE_API_KEY to your .env file")
    print("   Get your key at: https://aistudio.google.com/app/apikey")

# ============================================================================
# Step 1c: Import Utils and Initialize LLM Client
# ============================================================================
try:
    from utils import setup_llm_client, get_completion, save_artifact
    print("✓ Utils imported successfully")
except ImportError as e:
    print(f"❌ ERROR importing utils: {e}")
    raise

# Check for google-genai package
try:
    import google.genai
    print("✓ google-genai package is installed")
except ImportError:
    print("❌ ERROR: google-genai is not installed!")
    print("   Run: pip install google-genai")
    raise

# ============================================================================
# Step 1d: Initialize the LLM Client
# ============================================================================
print("\n" + "="*60)
print("Initializing LLM Client...")
print("="*60)

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-flash")

if client is not None and api_provider is not None:
    print(f"\n✅ SUCCESS! LLM Client initialized")
    print(f"   Provider: {api_provider}")
    print(f"   Model: {model_name}")
else:
    print("\n❌ FAILED to initialize LLM client!")
    print("   Please check the error messages above.")
    print("\nCommon issues:")
    print("   1. Missing dependencies: Run 'pip install -r requirements.txt'")
    print("   2. Missing .env file: Create .env with GOOGLE_API_KEY")
    print("   3. Invalid API key: Check your key at https://aistudio.google.com/app/apikey")
    raise RuntimeError("LLM client initialization failed")


2025-11-04 11:51:23,618 ag_aisoftdev.utils WARNING Optional core dependencies not found. Some features will be degraded. provider=None model=None latency_ms=None artifacts_path=None
2025-11-04 11:51:23,621 ag_aisoftdev.utils WARNING To enable full functionality run: pip install python-dotenv ipython plantuml provider=None model=None latency_ms=None artifacts_path=None


📁 Project root: c:\Users\640109\OneDrive - BOOZ ALLEN HAMILTON\Documents\AISE_Capstone\220372-AG-AISOFTDEV-Team-1-AINavigators
✓ python-dotenv is installed
✓ Loaded .env file from: c:\Users\640109\OneDrive - BOOZ ALLEN HAMILTON\Documents\AISE_Capstone\220372-AG-AISOFTDEV-Team-1-AINavigators\.env
✓ GOOGLE_API_KEY found: AIzaSyBZ..._C7Q
✓ Utils imported successfully


2025-11-04 11:51:31,016 ag_aisoftdev.utils WARNING python-dotenv not installed; .env will not be loaded. provider=None model=None latency_ms=None artifacts_path=None


✓ google-genai package is installed

Initializing LLM Client...


2025-11-04 11:51:33,671 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-flash latency_ms=None artifacts_path=None



✅ SUCCESS! LLM Client initialized
   Provider: google
   Model: gemini-2.5-flash


## Step 2: Define the Problem Statement

Our project is **TripSplit**, a group travel expense manager that helps travelers track shared expenses and calculate settlements.


In [5]:
project_idea = """
TripSplit: Group Travel Expense Manager

A collaborative expense tracking application specifically designed for group travel. 
Travelers can create trips, add participants, log expenses, and automatically calculate 
who owes whom at the end of the trip.

Core Features:
- Create and manage trips with multiple participants
- Add expenses with category, amount, payer, and split method (equal, percentage, custom)
- View real-time balance summary showing who owes whom
- Generate settlement recommendations (minimize number of transactions)
- Export expense reports
- RAG: Chat interface to query expenses ("How much did we spend on food?")
- Agent: Smart categorization of expenses using AI

Example user flow:
a person makes an account
in the account, you are able to create multiple trips
inside a particular trip, you can add participants in the trip by name, there is no need for a participant to have an account. 
in the trip, you have a total expense that is itemized which you can specify who paid for the total expense
within each expense, you can choose how to split up the cost i.e. evenly, per item, percentages
 
the end result is a view of who paid for what, and who owes what to who
"""

problem_statement = """We need a tool to help groups of travelers manage shared expenses during trips, 
track who paid for what, and automatically calculate fair settlements at the end of the journey."""

print("Project Idea:")
print(project_idea)
print("\nProblem Statement:")
print(problem_statement)


Project Idea:

TripSplit: Group Travel Expense Manager

A collaborative expense tracking application specifically designed for group travel. 
Travelers can create trips, add participants, log expenses, and automatically calculate 
who owes whom at the end of the trip.

Core Features:
- Create and manage trips with multiple participants
- Add expenses with category, amount, payer, and split method (equal, percentage, custom)
- View real-time balance summary showing who owes whom
- Generate settlement recommendations (minimize number of transactions)
- Export expense reports
- RAG: Chat interface to query expenses ("How much did we spend on food?")
- Agent: Smart categorization of expenses using AI

Example user flow:
a person makes an account
in the account, you are able to create multiple trips
inside a particular trip, you can add participants in the trip by name, there is no need for a participant to have an account. 
in the trip, you have a total expense that is itemized which you c

## Step 3: Brainstorm Features and User Personas

Use the LLM to expand on our idea by brainstorming detailed features and identifying key user personas.


In [7]:
# Brainstorm features
features_prompt = f"""
Based on the following project idea, brainstorm a comprehensive list of features for this application.
Organize them into logical categories and provide detailed descriptions.

Project Idea:
{project_idea}

Problem Statement:
{problem_statement}

Generate a detailed markdown list of features covering:
- Core expense management functionality
- Trip and participant management
- Settlement and balance calculations
- Reporting and export capabilities
- User experience enhancements
"""

print("=" * 80)
print("BRAINSTORMING FEATURES")
print("=" * 80)
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)
print("\n")


BRAINSTORMING FEATURES
Here's a comprehensive list of features for the TripSplit application, organized into logical categories with detailed descriptions:

---

### TripSplit: Group Travel Expense Manager - Feature List

#### I. Trip and Participant Management

*   **User Account Management**
    *   **Description:** Allows users to create personal accounts, log in securely, manage their profile information (name, email, password), and view all trips they are associated with.
*   **Trip Creation & Management**
    *   **Description:** Enable users to create new trips, specifying a trip name, dates (start/end), destination, and an optional description. Users can also edit existing trip details or delete trips they own.
*   **Participant Management**
    *   **Add Participants by Name:** Users can add participants to a trip simply by their name, without requiring the participant to have a TripSplit account. This is crucial for ease of use.
    *   **Invite Account Holders:** Option to i

In [8]:
# Identify user personas
personas_prompt = f"""
Based on the following project idea, identify three distinct user personas who would use this application.
For each persona, provide:
- A descriptive name
- Their background and role
- Their goals and motivations
- Their pain points when managing group travel expenses
- What they need from this tool

Project Idea:
{project_idea}

Problem Statement:
{problem_statement}

Format your response as detailed markdown with clear sections for each persona.
"""

print("=" * 80)
print("IDENTIFYING USER PERSONAS")
print("=" * 80)
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)
print("\n")


IDENTIFYING USER PERSONAS
Here are three distinct user personas for the TripSplit application:

---

### Persona 1: The Trip Treasurer

*   **Descriptive Name:** The Trip Treasurer
*   **Background and Role:** Sarah, 32, is the meticulous planner and organizer for her friend group's annual vacations. She enjoys researching destinations, booking flights and accommodation, and generally ensuring everything runs smoothly. She's tech-savvy and often ends up paying upfront for major group expenses like accommodation, car rentals, or large grocery hauls. Her role naturally extends to managing the group's finances, aiming for fairness and efficiency.
*   **Goals and Motivations:**
    *   To ensure everyone in the group contributes their fair share to shared expenses.
    *   To avoid any awkward money conversations or misunderstandings during or after the trip.
    *   To minimize the administrative burden of tracking expenses so she can also enjoy the vacation.
    *   To get reimbursed acc

## Step 4: Generate Structured User Stories (JSON)

Transform the brainstormed features and personas into formal Agile user stories with acceptance criteria in Gherkin format, output as JSON.


In [9]:
json_user_stories_prompt = f"""
Act as a Senior Product Manager. Using the brainstormed features and user personas as context,
generate a comprehensive list of user stories for the TripSplit application.

Each user story must:
- Follow the format: "As a [persona], I want to [action], so that [benefit]"
- Include detailed acceptance criteria in Gherkin format (Given/When/Then)
- Be specific and actionable

Brainstormed Features:
{brainstormed_features}

User Personas:
{user_personas}

Output Requirements:
Your response MUST be a valid JSON array of objects. Each object must have these exact keys:
- "id": A unique identifier (e.g., "US001", "US002")
- "user_story": The user story text
- "persona": The persona name
- "acceptance_criteria": An array of strings with Gherkin-style criteria

Generate at least 10-15 user stories covering all major features.

CRITICAL: Your response must begin with [ and end with ]. Do not include any text before or after the JSON array.
"""

print("=" * 80)
print("GENERATING USER STORIES AS JSON")
print("=" * 80)

json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Parse and validate the JSON
try:
    # Clean up markdown fences if present
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print(f"✓ Successfully parsed {len(user_stories_json)} user stories as JSON.\n")
    
    if user_stories_json:
        print("Sample User Story:")
        print(json.dumps(user_stories_json[0], indent=2))
    else:
        print("⚠ Warning: JSON array is empty.")

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"✗ Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str[:500])
    user_stories_json = []


GENERATING USER STORIES AS JSON
✓ Successfully parsed 14 user stories as JSON.

Sample User Story:
{
  "id": "US001",
  "user_story": "As a Trip Treasurer, I want to create a new trip and easily add participants by their names, so that I can quickly set up our group's expense tracking without requiring everyone to sign up immediately.",
  "persona": "The Trip Treasurer",
  "acceptance_criteria": [
    "Given I am logged into TripSplit",
    "When I click 'Create New Trip', enter 'Europe Adventure', specify dates, and add 'Sarah', 'David', 'Emily' by name",
    "Then a new trip 'Europe Adventure' is created with Sarah, David, and Emily as participants",
    "And I am designated as the trip owner/admin"
  ]
}


## Step 5: Validate and Save User Stories

Programmatically validate the structure of our user stories and save them as a JSON artifact.


In [10]:
def validate_and_save_stories(stories_data, output_path="Artifacts/Documentation/user_stories.json"):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("✗ Validation Failed: Data is not a non-empty list.")
        return False

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    for idx, story in enumerate(stories_data):
        # Check if the story is a dictionary
        if not isinstance(story, dict):
            print(f"✗ Validation Error: Story at index {idx} is not a dictionary.")
            all_stories_valid = False
            continue
        
        # Check if all required keys are present
        missing_keys = [key for key in required_keys if key not in story]
        if missing_keys:
            print(f"✗ Validation Error: Story at index {idx} (ID: {story.get('id', 'Unknown')}) is missing keys: {missing_keys}")
            all_stories_valid = False
            continue
        
        # Check if the acceptance_criteria list is not empty
        if not isinstance(story['acceptance_criteria'], list) or len(story['acceptance_criteria']) == 0:
            print(f"✗ Validation Error: Story at index {idx} (ID: {story['id']}) has empty or invalid acceptance_criteria.")
            all_stories_valid = False
            continue

    if all_stories_valid:
        print(f"✓ All {len(stories_data)} user stories passed validation.\n")
        
        # Save the artifact
        save_artifact(json.dumps(stories_data, indent=2), output_path)
        print(f"✓ User stories successfully saved to {output_path}")
        return True
    else:
        print("\n✗ Validation failed. Artifact not saved.")
        return False

# Validate and save
print("=" * 80)
print("VALIDATING AND SAVING USER STORIES")
print("=" * 80)

if user_stories_json:
    validation_success = validate_and_save_stories(user_stories_json)
else:
    print("⚠ Skipping validation as user_stories_json is empty or not defined.")
    validation_success = False


VALIDATING AND SAVING USER STORIES
✓ All 14 user stories passed validation.

✓ User stories successfully saved to Artifacts/Documentation/user_stories.json


## Step 6: Generate Product Requirements Document (PRD)

Now we'll use the validated user stories to generate a comprehensive PRD. We'll create a custom template structure for TripSplit.


In [11]:
# Define a PRD template structure for TripSplit
prd_template_structure = """
# Product Requirements Document Structure

## 1. Executive Summary & Vision
- Product name, overview, purpose, and vision

## 2. The Problem
- Problem statement
- User personas and scenarios

## 3. Goals & Success Metrics
- Table with Goal, KPI, and Target columns

## 4. Functional Requirements & User Stories
- Organized by Epic
- User stories with acceptance criteria

## 5. Non-Functional Requirements (NFRs)
- Performance
- Security
- Accessibility
- Scalability
- Usability
- Reliability

## 6. Technical Considerations
- Technology stack recommendations
- Database design considerations
- API design principles

## 7. Release Plan & Milestones
- MVP features
- Future versions

## 8. Out of Scope & Future Considerations
- Out of scope for V1.0
- Future work

## 9. Appendix & Open Questions
- Open questions
- Dependencies and assumptions
"""

prd_generation_prompt = f"""
You are a Senior Product Manager tasked with creating a comprehensive Product Requirements Document (PRD)
for TripSplit, a group travel expense management application.

Project Context:
{project_idea}

User Stories:
{json.dumps(user_stories_json, indent=2)}

User Personas:
{user_personas}

Instructions:
1. Create a comprehensive PRD following this structure:
{prd_template_structure}

2. Use the provided user stories to populate the "Functional Requirements & User Stories" section
3. Group user stories into logical Epics (e.g., "Trip Management", "Expense Tracking", "Settlement Calculation")
4. For sections not covered in user stories (Success Metrics, NFRs, Technical Considerations, Release Plan),
   use your expertise to create logical and appropriate content
5. Make the PRD specific to TripSplit - avoid generic content
6. Ensure all sections are comprehensive and actionable
7. Format as clean, professional markdown

Generate the complete PRD now.
"""

print("=" * 80)
print("GENERATING PRODUCT REQUIREMENTS DOCUMENT")
print("=" * 80)

if user_stories_json:
    prd_output = get_completion(prd_generation_prompt, client, model_name, api_provider, temperature=0.3)
    print("✓ PRD generated successfully.\n")
    print("Preview (first 1000 characters):")
    print(prd_output[:1000])
    print("\n... (full document will be saved to file)")
else:
    print("⚠ Skipping PRD generation because user stories are missing.")
    prd_output = ""


GENERATING PRODUCT REQUIREMENTS DOCUMENT
✓ PRD generated successfully.

Preview (first 1000 characters):
# Product Requirements Document: TripSplit - Group Travel Expense Manager

## 1. Executive Summary & Vision

**Product Name:** TripSplit

**Overview:** TripSplit is a collaborative expense tracking application meticulously designed to simplify financial management for group travel. It empowers travelers to effortlessly create trips, add participants (even those without an account), log expenses with flexible splitting options, view real-time balances, and generate optimized settlement recommendations. By leveraging AI for smart categorization and a RAG-powered chat interface for quick insights, TripSplit aims to eliminate the financial friction often associated with group trips, fostering transparency and harmony among travelers.

**Purpose:** The primary purpose of TripSplit is to provide a seamless, transparent, and efficient solution for managing shared expenses during group trav

## Step 7: Save the PRD

Save the generated PRD to our artifacts folder.


In [12]:
if prd_output:
    prd_path = "Artifacts/Documentation/prd.md"
    save_artifact(prd_output, prd_path)
    print(f"✓ PRD successfully saved to {prd_path}")
else:
    print("⚠ No PRD to save.")


✓ PRD successfully saved to Artifacts/Documentation/prd.md


## Step 8: Generate Pydantic Validation Model

Create a Pydantic model to programmatically validate PRD structure in the future.


In [13]:
pydantic_model_prompt = f"""
You are a Python developer creating a Pydantic model for validating Product Requirements Documents.

Based on this PRD structure:
{prd_template_structure}

Requirements:
1. Create a Python class named 'ProductRequirementsDocument' that inherits from Pydantic's BaseModel
2. Add fields for each major section of the PRD
3. Use appropriate Python types: str, List, Dict, Optional, etc.
4. For user stories, create a nested UserStory model with fields: id, persona, user_story, acceptance_criteria
5. For goals/metrics, create a nested GoalMetric model with fields: goal, kpi, target
6. Add descriptive docstrings with Field(..., description="...") for each field
7. Import all necessary modules (pydantic, typing, datetime if needed)
8. Make the model comprehensive and production-ready

Generate ONLY the Python code without markdown formatting. The code should be ready to save as a .py file.
"""

print("=" * 80)
print("GENERATING PYDANTIC VALIDATION MODEL")
print("=" * 80)

pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider, temperature=0.2)

# Clean up markdown fences if present
if '```' in pydantic_model_code:
    pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()

print("✓ Pydantic model generated.\n")
print("Preview (first 800 characters):")
print(pydantic_model_code[:800])
print("\n... (full model will be saved to file)")


GENERATING PYDANTIC VALIDATION MODEL
✓ Pydantic model generated.

Preview (first 800 characters):
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

class UserStory(BaseModel):
    """
    Represents a single user story within the PRD.
    """
    id: str = Field(..., description="Unique identifier for the user story (e.g., US-001).")
    persona: str = Field(..., description="The user persona performing the action (e.g., 'As a registered user').")
    user_story: str = Field(..., description="The user story itself (e.g., 'I want to log in so that I can access my dashboard').")
    acceptance_criteria: List[str] = Field(..., description="List of acceptance criteria that define when the user story is complete.")

class GoalMetric(BaseModel):
    """
    Represents a strategic goal with its associated Key Performance Indicator (KPI) and target.
    """
    goa

... (full model will be saved to file)


## Step 9: Save the Pydantic Model

Save the validation model for future use.


In [14]:
if pydantic_model_code:
    model_path = "Artifacts/Documentation/prd_validation_model.py"
    save_artifact(pydantic_model_code, model_path)
    print(f"✓ Pydantic model successfully saved to {model_path}")
else:
    print("⚠ No Pydantic model to save.")


✓ Pydantic model successfully saved to Artifacts/Documentation/prd_validation_model.py


## Summary: Phase 1 Complete! 🎉

We have successfully completed Phase 1 of the capstone project. Here's what we've generated:

### Artifacts Created:
1. **User Stories (JSON)**: `Artifacts/Documentation/user_stories.json`
   - Structured, validated user stories with acceptance criteria
   - Machine-readable format for downstream processing

2. **Product Requirements Document**: `Artifacts/Documentation/prd.md`
   - Comprehensive PRD with all major sections
   - Executive summary, problem statement, user personas
   - Functional and non-functional requirements
   - Release plan and technical considerations

3. **Pydantic Validation Model**: `Artifacts/Documentation/prd_validation_model.py`
   - Programmatic validation for PRD structure
   - Ensures consistency across documentation

### Next Steps:
- **Phase 2**: Use the PRD to generate system architecture and database schema
- **Phase 3**: Build the FastAPI backend
- **Phase 4**: Create tests and perform security review
- **Phase 5**: Build the React frontend

### Key Takeaways:
- LLMs can transform vague ideas into structured, actionable requirements
- JSON output enables automated processing and validation
- Template-driven generation ensures consistency
- Programmatic validation adds reliability to AI-generated artifacts
